In [1]:
import json
import sys
import torch
import os
import pickle, gzip
import json
import pandas as pd
import itertools
# !pip install --upgrade orjson
import orjson
sys.path.insert(0, '/home/sulcan/Documents/ipac-logbook/code/')
sys.path.insert(0, '/home/sulcan/Documents/chatqcd/code/')
from mmd import *
from llm_chatqcd import *

from glob import glob
from tqdm.notebook import tqdm
from pprint import pprint
# from trl import SFTConfig, SFTTrainer
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
from unsloth import FastLanguageModel
from datasets import Dataset, load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import os 
os.environ['WANDB_DISABLED'] = 'true'
# model_id = 'mistralai/Mistral-7B-Instruct-v0.2'
# model_id = 'unsloth/Phi-3-medium-4k-instruct'
# model_id = 'microsoft/Phi-3-mini-4k-instruct'
model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"
# device = torch.device('cuda:2')

gd = lambda x,i : x[list(x.keys())[i]]
data_folder = '/home/sulcan/Documents/chatqcd/data/'

# !pip install xformers
# !pip install trl
# !pip install peft
# !pip install "unsloth[cu126-torch240] @ git+https://github.com/unslothai/unsloth.git"

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
# model_id = '/home/sulcan/Documents/chatqcd/models/unsloth/'

### Preparing model

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(model_name=model_id, 
                                                     dtype = None, 
                                                     load_in_4bit=True)
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 8,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

'''
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj","o_proj"]
)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)                                         # bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             device_map = device, 
                                             quantization_config = quantization_config,
                                             attn_implementation="eager")

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.gradient_checkpointing_enable()
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
# tokenizer.add_bos_token, tokenizer.add_eos_token

model.print_trainable_parameters()
'''


==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-PCIE-40GB. Max memory: 39.496 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


'\npeft_config = LoraConfig(\n    r=16,\n    lora_alpha=32,\n    lora_dropout=0.05,\n    bias="none",\n    task_type="CAUSAL_LM",\n    target_modules=["q_proj", "k_proj", "v_proj","o_proj"]\n)\n\nquantization_config = BitsAndBytesConfig(\n    load_in_4bit=True,\n    bnb_4bit_quant_type="nf4",\n    bnb_4bit_use_double_quant=True,\n    bnb_4bit_compute_dtype=torch.bfloat16\n)                                         # bnb_4bit_compute_dtype=torch.bfloat16)\n\nmodel = AutoModelForCausalLM.from_pretrained(model_id, \n                                             device_map = device, \n                                             quantization_config = quantization_config,\n                                             attn_implementation="eager")\n\ntokenizer = AutoTokenizer.from_pretrained(model_id)\n\nmodel = prepare_model_for_kbit_training(model)\nmodel = get_peft_model(model, peft_config)\n\nmodel.config.use_cache = False # silence the warnings. Please re-enable for inference!\nmodel.gradi

### Unsupervised pretraining

In [4]:
list_unsupervised = []

with gzip.open(f'{data_folder}/arxiv_with_prompts.pickle.gzip','rb') as f:
    data_unsupervised = pickle.load(f)
    
    for paper_k in tqdm(data_unsupervised):
        # raw_unupservised[paper_k] = data_unsupervised[paper_k]['preprocessed']
        for title, section in data_unsupervised[paper_k]['sections']:
            if 'References' in title:
                continue
            if len(section) < 1024 or len(section) > 65535:
                continue
            # section = re.sub(r'#+\s*\d*\s*','',section)
            section = re.sub(r'#+\s*.*?\n','',section)
            
            '''
            input_ = tokenizer.apply_chat_template(
                [{'role' : 'user', 'content' : ''}, 
                 {'role' : 'assistant', 'content' : section}],tokenize=False)
            '''
            input_ = section 
            list_unsupervised.append({'input' : input_,
                                      'paper_id' : paper_k})

  0%|          | 0/45422 [00:00<?, ?it/s]

### Supervised Dataset

Loading Q&A Data for tranining

In [5]:
qas = defaultdict(lambda : [])

for file in tqdm(glob(f'{data_folder}/qas_*.pickle')):
    with open(file,'rb') as f:
        try:
            qas_ = pickle.load(f)
            qas.update({k : qas_[k] for k in qas_ if len(qas_[k]) > 0})
        except Exception as e:
            print(f'{file} corrputed {str(e)}')

  0%|          | 0/9 [00:00<?, ?it/s]

Compiling LLM outputs to Q&A and generating training pairs

In [6]:
list_supervised = []

for paper_k in tqdm(qas):
    eq_dict = build_eq_tag_dict(data_unsupervised[paper_k]['raw'])
    for chapter_idx in range(len(qas[paper_k])):
        llm_output = qas[paper_k][chapter_idx]['output'][0]
        llm_output = llm_output.split('[/INST]')[-1].split('</s>')[0]
        try:
            res = extract_qa(llm_output.replace('\\','\\\\'), paper_k, chapter_idx)
            res = append_equations_to_question_answers(res,eq_dict)
            list_supervised.extend(res)
        except Exception as e:
            # print(f'Error {str(e)}')
            continue

  0%|          | 0/38613 [00:00<?, ?it/s]

In [7]:
for i in tqdm(range(len(list_supervised))):
    q = list_supervised[i]['question']
    a = list_supervised[i]['answer']
    list_supervised[i]['input'] = prompt_template(tokenizer, q, a)

  0%|          | 0/2066903 [00:00<?, ?it/s]

### Training

In [8]:
skip = 1
dataset = Dataset.from_list(list_unsupervised[::skip]  + list_supervised[::skip])
dataset = dataset.shuffle(seed = 42)

print(f'\n\n\n{skip}\n\n\n')




1





In [9]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "input",
    max_seq_length = model.max_seq_length,
    dataset_num_proc = 16,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8,
        warmup_steps = 16,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.005,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "../models/results/",
        save_steps = 10,
    ),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Map (num_proc=16):   0%|          | 0/2462571 [00:00<?, ? examples/s]

In [11]:
trainer_stats = trainer.train(resume_from_checkpoint = '/home/sulcan/Documents/chatqcd/models/results/checkpoint-31380/')# resume_from_checkpoint = False)

/home/sulcan/.local/lib/python3.11/site-packages/transformers/trainer.py:3098: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NA

Step,Training Loss
31381,1.172700
31382,1.519800
31383,1.575700
31384,1.311100
31385,1.368900
31386,1.440400
31387,1.461100
31388,1.444000
31389,1.586900
31390,1.489200


KeyboardInterrupt: 

In [ ]:
from transformers import TextStreamer

# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer("What is QCD Lagrangian, can you write it in LATEX?", return_tensors = "pt").to("cuda")
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

In [ ]:
model.save_pretrained("../models/unsloth/") # Local saving
tokenizer.save_pretrained("../models/unsloth/")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving